# Import dependencies

In [1]:
#import dependencies 
import pandas as pd
import geopandas as gpd
import numpy
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from sqlalchemy import create_engine

# Load in Chicago Crimes data and clean

In [2]:
#load in file
file_to_load = "Resources/chicago_crimes.csv"

#store into pandas data frame
crime = pd.read_csv(file_to_load)

In [3]:
crime.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,JE145602,12/22/2020 08:59:00 PM,004XX W 58TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,N,N,711,20.0,11,NaN,NaN,NaN,NaN,NaN
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)"
2,JE145783,04/01/2020 12:00:00 PM,027XX S ARCHER AVE,0820,THEFT,$500 AND UNDER,STREET,N,N,913,11.0,06,NaN,NaN,NaN,NaN,NaN
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)"


In [4]:
crime.dropna(inplace = True)

In [5]:
crime.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)"
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)"
5,JD381795,09/13/2020 12:00:00 PM,015XX N KINGSBURY ST,0810,THEFT,OVER $500,GROCERY FOOD STORE,Y,N,1822,27.0,06,1169505.0,1910252.0,41.909240,-87.652748,"(41.909240284, -87.65274758)"
6,JD219381,04/28/2020 07:15:00 PM,085XX S GIVINS CT,1330,CRIMINAL TRESPASS,TO LAND,RESIDENCE - YARD (FRONT / BACK),N,N,622,21.0,26,1172711.0,1848282.0,41.739118,-87.642800,"(41.739118468, -87.642799939)"


# Comma separate and combine latitude and longitude

In [6]:
crime["lat_long"]= crime["LATITUDE"].apply(lambda x:str(x)) +","+ crime["LONGITUDE"].apply(lambda x:str(x))

In [7]:
crime.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,lat_long
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)","41.796167139,-87.70951943"
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)","41.915454333,-87.638727274"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)","41.974129858,-87.766357256"
5,JD381795,09/13/2020 12:00:00 PM,015XX N KINGSBURY ST,0810,THEFT,OVER $500,GROCERY FOOD STORE,Y,N,1822,27.0,06,1169505.0,1910252.0,41.909240,-87.652748,"(41.909240284, -87.65274758)","41.909240284,-87.65274758"
6,JD219381,04/28/2020 07:15:00 PM,085XX S GIVINS CT,1330,CRIMINAL TRESPASS,TO LAND,RESIDENCE - YARD (FRONT / BACK),N,N,622,21.0,26,1172711.0,1848282.0,41.739118,-87.642800,"(41.739118468, -87.642799939)","41.739118468,-87.642799939"


# Create a function using "BEAT" to determine Zipcode

In [8]:
crime.groupby('BEAT').first()['lat_long']

BEAT
111     41.883500187,-87.627876698
112     41.881683753,-87.624353027
113     41.877578503,-87.629282088
114     41.887705775,-87.617022196
121      41.879331765,-87.63824099
                   ...            
2531    41.904615737,-87.773022962
2532     41.90834437,-87.763383589
2533    41.904159474,-87.747044455
2534    41.903446578,-87.734399671
2535    41.909907002,-87.724577987
Name: lat_long, Length: 274, dtype: object

In [9]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

In [10]:
def get_zipcode(lat_long):
    location = geolocator.geocode(lat_long)
    zip_code = location.raw['display_name'].split(",")[-2].strip()
    return zip_code

In [11]:
%%time
beats_zipcodes = crime.groupby('BEAT').first()['lat_long'].apply(get_zipcode)

CPU times: user 5.4 s, sys: 662 ms, total: 6.06 s
Wall time: 2min 17s


In [ ]:
beats_zipcodes.name="Zipcode"

In [23]:
beats_zipcodes

BEAT
111     60602
112     60603
113     60604
114     60601
121     60606
        ...  
2531    60651
2532    60651
2533    60651
2534    60651
2535    60302
Name: Zipcode, Length: 274, dtype: object

# Merge new column to existing DataFrame

In [24]:
crime_zipcode = crime.merge(pd.DataFrame(beats_zipcodes).reset_index(),on="BEAT")
crime_zipcode.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,lat_long,Zipcode
0,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)","41.796167139,-87.70951943",60632
1,JD368147,05/06/2020 07:00:00 PM,035XX W 57TH ST,0820,THEFT,$500 AND UNDER,APARTMENT,N,N,822,23.0,06,1153830.0,1866591.0,41.789756,-87.711492,"(41.789756197, -87.711492062)","41.789756197,-87.711492062",60632
2,JD455983,12/11/2020 06:00:00 AM,056XX S SAWYER AVE,0820,THEFT,$500 AND UNDER,STREET,N,N,822,14.0,06,1155678.0,1867012.0,41.790875,-87.704705,"(41.790874602, -87.704704643)","41.790874602,-87.704704643",60632
3,JD364284,09/11/2020 04:20:00 PM,052XX S LAWNDALE AVE,0485,BATTERY,AGGRAVATED OF A CHILD,RESIDENCE,N,N,822,23.0,04B,1152596.0,1869582.0,41.797988,-87.715938,"(41.797988346, -87.71593806)","41.797988346,-87.71593806",60632
4,JD364146,09/11/2020 05:00:00 PM,032XX W 55TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,N,Y,822,14.0,08B,1155567.0,1867964.0,41.793489,-87.705086,"(41.793489253, -87.705086148)","41.793489253,-87.705086148",60632


In [50]:
crime_zipcode.dropna(inplace=True)

# Extract relevant columns for analysis

In [49]:
crime_zipcode = crime_zipcode[["CASE#", "BEAT", "Zipcode"]]
crime_zipcode.head()

,CASE#,BEAT,Zipcode
0,JD220983,822,60632
1,JD368147,822,60632
2,JD455983,822,60632
3,JD364284,822,60632
4,JD364146,822,60632


# Load in Chicago Public Schools data and clean

In [35]:
#load in file
school_file = "Resources/public_schools.csv"

#store into pandas data frame
public_schools = pd.read_csv(school_file)

In [36]:
public_schools.head()

,School ID,Name of School,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,RCDTS Code,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,Community Area Number,Community Area Name,Ward,Police District,Location
0,609966,Charles G Hammond Elementary School,ES,2819 W 21st Pl,Chicago,IL,60623,(773) 535-4580,http://schoolreports.cps.edu/SchoolProgressRep...,Pilsen-Little Village Elementary Network,...,150000000000000,1157809.015,1889554.623,41.852691,-87.696278,30,SOUTH LAWNDALE,12,10,"(41.85269133, -87.69627777)"
1,610539,Marvin Camras Elementary School,ES,3000 N Mango Ave,Chicago,IL,60634,(773) 534-2960,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,150000000000000,1137482.296,1919394.900,41.934966,-87.770165,19,BELMONT CRAGIN,30,25,"(41.93496641, -87.77016525)"
2,609852,Eliza Chappell Elementary School,ES,2135 W Foster Ave,Chicago,IL,60625,(773) 534-2390,http://schoolreports.cps.edu/SchoolProgressRep...,Ravenswood-Ridge Elementary Network,...,150000000000000,1161016.902,1934466.545,41.975867,-87.683254,4,LINCOLN SQUARE,47,20,"(41.975867, -87.68325438)"
3,609835,Daniel R Cameron Elementary School,ES,1234 N Monticello Ave,Chicago,IL,60651,(773) 534-4290,http://schoolreports.cps.edu/SchoolProgressRep...,Garfield-Humboldt Elementary Network,...,150000000000000,1151767.546,1908130.068,41.903785,-87.717963,23,HUMBOLDT PARK,26,25,"(41.90378521, -87.71796315)"
4,610521,Sir Miles Davis Magnet Elementary Academy,ES,6730 S Paulina St,Chicago,IL,60636,(773) 535-9120,http://schoolreports.cps.edu/SchoolProgressRep...,Englewood-Gresham Elementary Network,...,150000000000000,1166133.893,1859929.018,41.771222,-87.666567,67,WEST ENGLEWOOD,15,7,"(41.77122181, -87.66656657)"


# Extract relevant columns for analysis

In [45]:
schools = public_schools[['Name of School',
                                'ZIP Code', 'CPS Performance Policy Level',
                                'Safety Score', 'Family Involvement Score',
                                'Parent Engagement Score', 'Average Student Attendance']]
schools.head()

,Name of School,ZIP Code,CPS Performance Policy Level,Safety Score,Family Involvement Score,Parent Engagement Score,Average Student Attendance
0,Charles G Hammond Elementary School,60623,Level 2,40.0,NDA,43,95.3
1,Marvin Camras Elementary School,60634,Not Enough Data,54.0,58,51,95.1
2,Eliza Chappell Elementary School,60625,Level 1,70.0,65,50,95.1
3,Daniel R Cameron Elementary School,60651,Level 3,42.0,NDA,46,92.6
4,Sir Miles Davis Magnet Elementary Academy,60636,Level 3,35.0,NDA,NDA,93.0


# Connect to local database

In [38]:
username = "postgres"
password = "9229mango"

In [39]:
rds_connection_string = f"{username}:{password}@localhost:5432/crime"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Check for tables

In [40]:
engine.table_names()

<ipython-input-40-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['crime', 'schools']

# Use Pandas to load csv converted DataFrame into database

In [43]:
crime_zipcode.to_sql(name='crime', con=engine, if_exists='append', index=False)

In [46]:
schools.to_sql(name='schools', con=engine, if_exists='append', index=False)

# Confirm data has been added by querying the crime table

In [47]:
pd.read_sql_query('select * from crime', con=engine).head()

,CASE#,BEAT,Zipcode
0,JD220983,822,60632
1,JD368147,822,60632
2,JD455983,822,60632
3,JD364284,822,60632
4,JD364146,822,60632


# Confirm data has been added by querying the schools table

In [51]:
pd.read_sql_query('select * from schools', con=engine).head()

,Name of School,ZIP Code,CPS Performance Policy Level,Safety Score,Family Involvement Score,Parent Engagement Score,Average Student Attendance
0,Charles G Hammond Elementary School,60623,Level 2,40.0,NDA,43,95.3
1,Marvin Camras Elementary School,60634,Not Enough Data,54.0,58,51,95.1
2,Eliza Chappell Elementary School,60625,Level 1,70.0,65,50,95.1
3,Daniel R Cameron Elementary School,60651,Level 3,42.0,NDA,46,92.6
4,Sir Miles Davis Magnet Elementary Academy,60636,Level 3,35.0,NDA,NDA,93.0
